In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

def cubicarray(MM, pr=False):
    if pr: print('numero di elementi: ', len(MM))
    # In input bisogna fornire la lista dei monomi M. Ogni elemento della lista M e'
    # una lista [a,b,c] tale per cui M_i=x^{2a}y^{2b}z^{2c} e a+b+c=n, dove n e' il grado del monomio.
    # I monomi in input comprendono tutte le permutazioni.
    # Voglio ora applicare le permutazioni x,y,z in modo da simmetrizzare i monomi e ottenere le armoniche cubiche.
    lenM = len(MM)
    M=[]
    #Converto M a una lista di liste
    for i in range(len(MM)):
        M.append(list(MM[i]))
    Mt = copy.copy(M)
    # devo prima capire nella base dei monomi quali sono legati da permutazione. posso nominarli.
    nameC = []
    conta = 0
    contaci = 0
    if pr: print('numero cicli teornicamente necessari', int(len(M) ** 2 * 3 / 2))
    for i in range(lenM):
        rep = M[i]
        # rep (M[i]) e' il mio rappresentante nella lista,
        # voglio vedere quali altri elementi sono equivalenti per permutazione a rep
        nameC_i = []
        if not (rep in Mt): continue
        # se rep non sta nella lista temporanea Mt significa che e' equivalente a un elemento precedente: salto il ciclo
        nameC_i.append(np.array(rep))
        # in nameC_i inserisco gli elementi equivalenti per simmetria a rep
        for j in range(i + 1, lenM):
            # parto a cercare dall'elemento successivo nella lista, gli altri li ho gia' sicuramente controllati.
            Mjt = list(copy.copy(M[j]))
        
            if rep[0] in Mjt:
                Mjt.remove(rep[0])
                if rep[1] in Mjt:
                    Mjt.remove(rep[1])
                    if rep[2] in Mjt:
                        # rep e M[j] sono simmetrici per permutazione!
                        nameC_i.append(np.array(M[j]))
                        Mt.remove(M[j])
                        
                        # rimuovo da Mt l'elemento M[j] che e' simmetrico per permutazione a rep
                        conta += 1
        contaci += len(nameC_i)
        nameC.append(nameC_i)
        # aggiungo la lista degli elementi associati a M[i], nameC_i, alla lista degli elementi NON equivalenti, nameC!
    if contaci != len(M): raise ValueError
    if pr: print('numero cicli compiuti', conta)
    if pr: print('numero di elementi non equivalenti per simmetria cubica: ', len(nameC))
    # in output c'e' la lista degli elementi che formano una classe di equivalenza.
    return nameC

def datadicG(data, sigmadata, G):

    dicdata = {}
    
    dicsigma = {}
    
    for i in range(len(G)):
        dicdata[tuple(G[i])] = data[i]
        dicsigma[tuple(G[i])] = sigmadata[i]
        
    return dicdata, dicsigma

def datiplot(C, dic, dics):
    gplot=[]
    dataplot=[]
    datasigmaplot=[]
    for i in range(len(C)):
        gplot.append(C[i][0])
        dataav=0
        datasigma=0
        for j in range(len(C[i])):
            dataav+=dic[tuple(C[i][j])]/len(C[i])  
            datasigma+=(dics[tuple(C[i][j])]/len(C[i])  )**2
        dataplot.append(dataav)
        datasigmaplot.append(np.sqrt(datasigma))
    
    return np.array(gplot), np.array(dataplot), np.array(datasigmaplot)



In [2]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 1. 0.] [1. 0. 0.] [0. 0. 1.]


In [3]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e ritorno la lista dei nuovi dati 'simmetrizzati'
#in gplot e sd
C=cubicarray(list(grid), True)
dic, dics=datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [4]:
#Calcolo il fit bayesiano con glot e sd come test
mN, SN, y_infer, sy_infer=bayesianL.bayesianpol(gplot,sd,12,100,0.001,gplot,True)

parametri ottimali [   0.49533012  -17.35055934   29.2230084   -66.74101773 -119.79136651
 -241.85766212 -117.80340288   55.00981355  -15.48742048   50.25328741
  -26.81453661   31.61263082  -22.48757593   38.72873981   38.85709273
   26.36044947  -18.81729691  -11.24980862  -15.09014409  -19.96185365
  -44.07462164  -11.20924265   -2.77822352   12.13151947   -0.34631563
   34.81649535   34.12029364   23.6083751   -17.10053123  -19.1295225
  -12.63368844]
numero di armoniche cubiche 19
valore a kmin inferito 0.4654349922988382 bias 0.49533011911271385 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 5.943383058344656e+36


In [5]:
#Calcolo il best fit su gplot e sd con 30 dati
N=30
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(gplot, sd, N, gplot, False, True)

parametri ottimali [  0.79176053 -16.09632273  30.83333646 -54.23554706 -16.22712376
 -69.30445026  -0.54922101   7.65930891 -57.16276544 -20.23968473
  -3.20628759 -58.79257311  -2.75245261  21.78089104  29.22622994
  20.18467849  -2.46323606   6.50787023  -7.11149174  -7.03450896
  -6.9542552   -5.90143271  -0.95716629  16.32893465  11.27049067
   0.56796526 -16.56573514  -8.39396127   0.19965894   0.87752371
   0.56380362]
numero di armoniche cubiche 19
valore a kmin inferito 0.6550801412616502 bias 0.791760531914077 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: -5.3852969149909954e-05
grado ottimale 24 grado massimo tentato 28


In [6]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#Plotto il best fit e i dati (gplot e sd)
C=cubicarray(list(grid), True)
dic, dics=datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [15]:
#Calcolo il best fit includendo sempre piu' punti 
predb=[]
spredb=[]
Npointsb=[]
r=0
for N in range(10, 114):
    
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=bayesianL.bestfit(gplot, sd, N+1, gplot, False, True)
    Npointsb.append(np.linalg.norm(gplot[N+1]))
    print(r, gplot[N+1], N+1)
    predb.append(mN[0])
    spredb.append(SN[0,0])

parametri ottimali [  0.9515267   -7.82839126  -2.32720451 -12.55498516  -0.90385436
   4.36891257  -0.68959113  -5.69299181  -8.9823982   -5.13545449]
numero di armoniche cubiche 5
valore a kmin inferito 0.7031454705614797 bias 0.9515267033727122 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 8.19287222516724e+20
grado ottimale 10 grado massimo tentato 28
0 [2. 2. 2.] 11


parametri ottimali [ 0.91889073 -7.6254733   4.99698944 -2.64716865 -0.24044129  0.06412747
  1.18158823 -3.34926583  1.03511065 -2.09716385 -1.70030783 -3.43900747
 -1.03142837]
numero di armoniche cubiche 7
valore a kmin inferito 0.696507754955347 bias 0.9188907312695247 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: -50529881.9597916
grado ottimale 12 grado massimo tentato 28
0 [3. 2. 0.] 12


parametri ottimali [ 0.82372922 -5.81312729  0.26041856 -4.82127358 -3.52927458 -2.2192458
 -1.86856391]
numero di armoniche cubiche 4
valore a kmin inferito 0.6687004521768652 bias 0.8237292212077235 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 2.0857217526800675e+19
grado ottimale 8 grado massimo tentato 28
0 [2. 1. 3.] 13


parametri ottimali [  0.92372019 -10.19391647   8.48812555 -11.73726531  -9.41252804
  -2.96084645  -4.87376536  -3.79767865  -5.71612235  -8.0879263
   2.82581532  -2.21386268   1.58942594]
numero di armoniche cubiche 7
valore a kmin inferito 0.695401498925662 bias 0.9237201935179012 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 1.0034115746803996e+18
grado ottimale 12 grado massimo tentato 28
0 [4. 0. 0.] 14


parametri ottimali [  0.87861801  -9.49590441   7.12163749 -13.01864365  -9.38255665
   6.64822043  -5.28398868  -0.54115996  -8.61425709  -3.68449031
   1.94677379  -6.42432659  -1.24661883  -0.78792787   0.95793356
  -0.48896302]
numero di armoniche cubiche 8
valore a kmin inferito 0.6831965018316141 bias 0.8786180131215038 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: -72353118684062.61
grado ottimale 14 grado massimo tentato 28
0 [1. 0. 4.] 15


...

parametri ottimali [   0.4391567   -12.40808552   15.23885454   20.82725938  -26.55578992
 -144.71022907  -26.65342826   20.16894332  -29.98193725    1.61839578
  -30.11712656   33.25748975  -12.47912053   21.52230567  -11.92071099
    9.89381831   -6.54790116    2.93501124   -2.67988888]
numero di armoniche cubiche 10
valore a kmin inferito 0.4207535140313321 bias 0.43915669877959546 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 4.915341990210079e+40
grado ottimale 16 grado massimo tentato 28
0 [2. 7. 7.] 110


parametri ottimali [   0.43819257  -12.4829028    14.42281707   21.64816849  -16.72274639
 -130.59865438  -20.70909365    6.26598409  -42.40746428  -10.46865293
  -11.24977451   34.52853146   -1.0715303     4.13043327   -6.50839016
    1.89060297]
numero di armoniche cubiche 8
valore a kmin inferito 0.4199721831349356 bias 0.4381925701607323 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 1.517120438695609e+39
grado ottimale 12 grado massimo tentato 28
0 [3. 7. 7.] 111


parametri ottimali [ 5.39701505e-01 -2.83561443e+01  5.68431381e+01  4.13430016e+01
 -4.63211671e+01 -8.98657362e+02 -1.54718419e+02 -2.94161525e+02
  2.17799767e+02 -1.09402618e+02  7.79610206e+02  3.06281458e+02
  3.96593737e+02 -6.52778380e+02 -1.16460819e+02 -3.31023236e+02
 -5.35710375e+01 -4.06077564e+02 -5.56231654e+01 -4.76255245e+02
  1.17605166e+02 -1.96270379e+02  1.50595876e+03  4.89596812e+02
  7.31609646e+02 -4.52719745e+02 -2.12804268e+02 -2.48180252e+02
 -6.34140700e+02 -1.83204988e+02 -2.85259725e+02  1.69289949e+01
 -1.79006442e+00  9.40412507e+00  2.37286429e+02  8.23581692e+01
  1.09622189e+02]
numero di armoniche cubiche 24
valore a kmin inferito 0.49991426303065173 bias 0.5397015050218591 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 8.581318372607618e+20
grado ottimale 20 grado massimo tentato 28
0 [6. 6. 6.] 112


parametri ottimali [ 5.19590321e-01 -2.57008482e+01  5.80466888e+01  7.02803451e+00
 -9.20940587e+01 -6.26994988e+02 -1.20807995e+02 -4.13355889e+01
  2.49408490e+01 -2.43142846e+01  3.44201485e+02  1.88486610e+02
  1.63167046e+02 -2.18732646e+02 -6.17972352e+01 -9.71955107e+01
 -6.28793570e+02 -1.37801859e+02 -2.83952795e+02  5.03732821e+02
  1.64925889e+02  2.39770552e+02  9.43763284e+02 -1.39427705e+01
  3.83745521e+02 -1.30107533e+03 -9.56836414e+01 -5.48353306e+02
  4.31387181e+02  5.01641941e+01  1.83892858e+02]
numero di armoniche cubiche 19
valore a kmin inferito 0.48423190835102486 bias 0.5195903210806626 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 1.1220917968569882e+30
grado ottimale 16 grado massimo tentato 28
0 [7. 5. 6.] 113


parametri ottimali [   0.48733591  -20.12023121   23.37778475   89.01662766    8.0818628
 -398.75054509  -20.57951742  -41.62896133 -191.68680416  -82.67756022
   30.06362763  206.46489889   42.31916536  -27.54265559  -32.06740702
   -9.18314357  -38.80854537  -43.87841498  -21.7462106    82.15180632
   17.74574404   33.65020582  -35.6166137    -2.25110405  -14.19208901]
numero di armoniche cubiche 14
valore a kmin inferito 0.45977017925922076 bias 0.48733590935165705 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 1.1790899571772167e+37
grado ottimale 16 grado massimo tentato 28
0 [4. 7. 7.] 114


In [16]:
#Calcolo il model averaging su gplot  e sd con 30 punti
N=20
mr, sr, logevidence= bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, False, True)

model average prediction 0.7872885136439942 0.013021111306251715


In [17]:
#Calcolo il model averaging includendo sempre piu' punti 
preda=[]
spreda=[]
Npointsa=[]
r=0
for N in range(10, 114):
    
    mr, sr, logevidence= bayesianL.bayesianmodelprediction(gplot, sd, N+1, gplot, False, True)
    Npointsa.append(np.linalg.norm(gplot[N+1]))
    print(r, gplot[N+1], N+1)
    preda.append(mr)
    spreda.append(sr)

model average prediction 0.5148256033159574 0.01648907684421719
0 [2. 2. 2.] 11


model average prediction 0.46630215587260937 0.014758902149765218
0 [3. 2. 0.] 12


model average prediction 0.42655017067821016 0.014536128189463456
0 [2. 1. 3.] 13


model average prediction 0.42596537493936615 0.01520373701006373
0 [4. 0. 0.] 14


model average prediction 0.38233282445488964 0.014256687605504989
0 [1. 0. 4.] 15


...

model average prediction 0.1077577543099497 0.0076124027951666616
0 [2. 7. 7.] 110


model average prediction 0.10638835740349985 0.007921568329606981
0 [3. 7. 7.] 111


model average prediction 0.10512141753744268 0.008110386763504578
0 [6. 6. 6.] 112


model average prediction 0.10416344699860568 0.00744471459183007
0 [7. 5. 6.] 113


model average prediction 0.09998813149139206 0.007771481364834194
0 [4. 7. 7.] 114


In [20]:
%matplotlib widget
fig, ax=plt.subplots(3,constrained_layout=True, figsize=[9,9])


ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[1].errorbar(np.array(Npointsb), np.array(predb), np.sqrt(np.array(spredb)), fmt='.')
ax[1].plot(np.array(Npointsb), np.ones(len(Npointsb))*0.816, label='GK result')    
ax[1].set_xlabel('$k_{max} (nm^{-1})$')
ax[1].set_ylabel('prediction at k=0')
ax[1].legend()
ax[2].set_title('Data')

ax[2].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[2].legend()

ax[2].set_xlabel(r'k ($nm^{-1}$)')
plt.savefig('nlttcubic.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …